In [68]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import glob
import re
from pathlib import Path
import cv2
import os,sys

In [2]:
from tqdm import tqdm

In [3]:
def bbox_yolo2voc(bx, w, h):
    x_c = float(bx[0])*w
    y_c = float(bx[1])*h
    wbbox = float(bx[2])*w
    hbbox = float(bx[3])*h
    x1 = int(x_c - wbbox/2)
    y1 = int(y_c - hbbox/2)
    x2 = int(x_c + wbbox/2)
    y2 = int(y_c + hbbox/2)
    bbox = [x1, y1, x2, y2]
    return bbox

In [4]:
def bbox_voc2yolo(bx, w, h):
    x1 = bx[0]
    y1 = bx[1]
    x2 = bx[2]
    y2 = bx[3]
    w_bbox = x2 - x1
    h_bbox = y2 - y1
    x_c = x1 + w_bbox/2
    y_c = y1 + h_bbox/2
    bbox = [x_c/w, y_c/h, w_bbox/w, h_bbox/h]
    return bbox

In [5]:
def bbox_coco2voc(bbox):
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]
    return bbox

In [6]:
def draw_bbox(img_np, box_coco):
    id_ = int(box_coco[0])
    x1 = int(box_coco[1])
    y1 = int(box_coco[2])
    x2 = int(box_coco[3])
    y2 = int(box_coco[4])
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.rectangle(img_np,(x1, y1),(x2, y2),(0,0,255),thickness=2)
    f = cv2.putText(img_np, text=str(id_), org=(x1 + 5, y1 + 5), fontFace=font, fontScale=1, 
        thickness=2, lineType=cv2.LINE_AA, color=(0, 255, 0))
    return img_np

In [7]:
def bboxes2string(bboxes):
    text = ""
    for bx in bboxes:
        text += str(int(bx[0]))
        for item in bx[1:]:
            text = text + ' ' + str(item)
        text += '\n'
    return text

def write_bboxes(p, bboxes_text):
    with open(p, 'w') as f:
        f.write(bboxes_text)

In [69]:
names = {0: ['小西装', 'Suit_xizhuang'],
 1: ['毛衣/绒衣/针织衫', 'Sweater/knit'],
 2: ['女衫类', 'Blouses '],
 3: ['皮衣', 'leather'],
 4: ['夹克/外套', 'Jacket/coat '],
 5: ['风衣', 'dust coat'],
 6: ['背心', 'vest'],
 7: ['吊带', 'harness'],
 8: ['牛仔', 'cowboy'],
 9: ['羽绒服', 'Down jacket'],
 10: ['大衣', 'coat'],
 11: ['皮草', 'fur'],
 12: ['卫衣', 'fleece'],
 13: ['棉衣/羊羔绒', 'Padded coat'],
 14: ['斗篷', 'cloak'],
 15: ['裙子', 'skirt'],
 16: ['西装裤', 'Suit pants'],
 17: ['牛仔裤', 'Jeans'],
 18: ['休闲裤', 'Slacks'],
 19: ['打底裤', 'Leggings'],
 20: ['运动裤', 'Sweatpants'],
 21: ['连衣裙', 'Dress'],
 22: ['连衣裤', 'Jumpsuit'],
 23: ['包', 'Bag'],
 24: ['鞋', 'Shoe']
 }

In [208]:
names_en = []
for k,v in names.items():
    names_en.append(v[1])

In [209]:
names_en

['Suit_xizhuang',
 'Sweater/knit',
 'Blouses ',
 'leather',
 'Jacket/coat ',
 'dust coat',
 'vest',
 'harness',
 'cowboy',
 'Down jacket',
 'coat',
 'fur',
 'fleece',
 'Padded coat',
 'cloak',
 'skirt',
 'Suit pants',
 'Jeans',
 'Slacks',
 'Leggings',
 'Sweatpants',
 'Dress',
 'Jumpsuit',
 'Bag',
 'Shoe']

### Show image.

In [8]:
ROOT = Path.cwd()
# ROOT = ROOT / 'deepedia/'
ROOT

PosixPath('/home/chenyi/workspace/myPyModule99')

In [ ]:
fname = '0000fe7c9191fba733c8a69cfaf962b7'
imgp = ROOT / 'images/train' /fname 

In [ ]:
Image.open(imgp)

In [ ]:
img_np = cv2.imread(str(p))
img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

In [ ]:
for bx in bboxes:
    bx[1:] = bbox_coco2voc(bx[1:])
    draw_bbox(img_np, bx)

In [ ]:
Image.fromarray(img_np)

### Dataset infomation

In [16]:
root = Path('/home/chenyi/workspace/dataset/deepedia_l2/')

In [25]:
!tree -L 2 $root

/home/chenyi/workspace/dataset/deepedia_l2
└── labels
    ├── train
    └── val

3 directories, 0 files


In [26]:
dpedia2_labels_dir = root /'labels/'

In [27]:
deepedia_root = '/home/chenyi/workspace/dataset/deepedia/info/'

In [29]:
!tree  -L 2 $deepedia_root

/home/chenyi/workspace/dataset/deepedia/info/
├── cat_pedia2ours.pickle
├── fashion2_id2name.pickle
├── fashion2info_train_pd.pickle
├── fashion2info_val_pd.pickle
├── pedia2yolo_train.pickle
├── pedia2yolo_val.pickle
├── pedia_info_pd.pickle
└── pedia_train_dict.pickle

0 directories, 8 files


In [70]:
with open(f'{deepedia_root}/pedia2yolo_train.pickle', 'rb') as f:
    pedia1_pd = pickle.load(f)

In [32]:
pedia1_pd.head()

,img_id,bbox,width,height,file_name,train,img_path,img_target,label_target
0,46226,"[[5.0, 0.544382801664355, 0.8759765625, 0.4563...",721,1024,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,train,/home/chenyi/workspace/dataset/data_fashionped...,/home/chenyi/workspace/dataset/deepedia/images...,/home/chenyi/workspace/dataset/deepedia/labels...
1,29130,"[[4.0, 0.50927734375, 0.544921875, 0.571289062...",1024,1024,0000fe7c9191fba733c8a69cfaf962b7.jpg,train,/home/chenyi/workspace/dataset/data_fashionped...,/home/chenyi/workspace/dataset/deepedia/images...,/home/chenyi/workspace/dataset/deepedia/labels...
2,32175,"[[7.0, 0.5058737151248165, 0.447265625, 0.2511...",681,1024,0002ec21ddb8477e98b2cbb87ea2e269.jpg,train,/home/chenyi/workspace/dataset/data_fashionped...,/home/chenyi/workspace/dataset/deepedia/images...,/home/chenyi/workspace/dataset/deepedia/labels...
4,38187,"[[7.0, 0.623900293255132, 0.60009765625, 0.543...",682,1024,0004467156e47b0eb6de4aa6479cbd15.jpg,train,/home/chenyi/workspace/dataset/data_fashionped...,/home/chenyi/workspace/dataset/deepedia/images...,/home/chenyi/workspace/dataset/deepedia/labels...
7,2501,"[[4.0, 0.5725806451612904, 0.6533203125, 0.784...",682,1024,000775b6b5e27b011dc8bba2d7b85211.jpg,train,/home/chenyi/workspace/dataset/data_fashionped...,/home/chenyi/workspace/dataset/deepedia/images...,/home/chenyi/workspace/dataset/deepedia/labels...


In [71]:
dpedia_catg23 =pd.read_csv('/nas/chenyi/dataset_apparel_nas/openfashion/deepedia_catg23.csv')

In [83]:
dpedia_catg23['fname'][-5:].values

array(['nas/lichangjian/open_fashion/Fashionpedia/train/fffc631acce2e28e1628de685d40c980.jpg',
       'nas/lichangjian/open_fashion/Fashionpedia/train/fffc631acce2e28e1628de685d40c980.jpg',
       'nas/lichangjian/open_fashion/Fashionpedia/train/fffe20b555b98c3c1f26c8dfff275cbc.jpg',
       'nas/lichangjian/open_fashion/Fashionpedia/train/fffe20b555b98c3c1f26c8dfff275cbc.jpg',
       'nas/lichangjian/open_fashion/Fashionpedia/train/ffffbf7014a9e408bfbb81a75bc70638.jpg'],
      dtype=object)

In [ ]:
re.findall(r'/(.*).jpg', str)

['this is first label</a>/<a>the second label</a>']

In [192]:
dpedia_catg23['fname'] = dpedia_catg23['path'].apply(lambda x: re.findall(r'/nas/.*/(.*?).jpg', x)[0])

In [197]:
dpedia_catg23.head()

,source,path,label,label_1,一级分类,label_2,二级分类,x_1,y_1,x_2,y_2,len,mode,fname
0,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,1.0,52.0,467.0,831.0,1,train,000003
1,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,0.0,113.0,467.0,623.0,1,train,000004
2,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,1.0,98.0,467.0,814.0,1,train,000005
3,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,0.0,324.0,466.0,831.0,1,train,000006
4,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,23.0,106.0,430.0,696.0,1,train,000007


In [199]:
x = dpedia_catg23[dpedia_catg23['len']==1]

In [200]:
len(x)

137589

In [185]:
pedia_lcj = dpedia_catg23[dpedia_catg23['source']=='Fashionpedia']

In [188]:
imgs_list = glob.glob('/home/chenyi/workspace/dataset/deepedia/images/*/*.jpg')

In [189]:
len(imgs_list)

269664

In [190]:
pedia_cy = pd.DataFrame()

In [191]:
pedia_cy['img_sp'] = imgs_list

In [196]:
pedia_cy.head()

,img_sp,fname
0,/home/chenyi/workspace/dataset/deepedia/images...,148589
1,/home/chenyi/workspace/dataset/deepedia/images...,102679
2,/home/chenyi/workspace/dataset/deepedia/images...,106021
3,/home/chenyi/workspace/dataset/deepedia/images...,131280
4,/home/chenyi/workspace/dataset/deepedia/images...,cbcc6cb04821176839d1e90a6f8f89b5


In [195]:
pedia_cy['fname'] =  pedia_cy['img_sp'].apply(lambda x: re.findall(r'/.*/(.*?).jpg', x)[0])

### fashion2 label.

In [45]:
of_df =pd.read_csv('/nas/chenyi/dataset_apparel_nas/openfashion/openfashion_info.csv')

In [66]:
# import os
# src = of_df.loc[345844]['path']
# dst = '1.jpg'
# os.symlink(src, dst)
# os.unlink(dst)

In [43]:
dpedia_catg23.head()

,source,path,label,label_1,一级分类,label_2,二级分类,x_1,y_1,x_2,y_2,len,mode
0,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,1.0,52.0,467.0,831.0,1,train
1,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,0.0,113.0,467.0,623.0,1,train
2,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,1.0,98.0,467.0,814.0,1,train
3,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,0.0,324.0,466.0,831.0,1,train
4,DeepFashion2,/nas/lichangjian/open_fashion/DeepFashion2/tra...,long sleeve dress,7,连体装,21,连衣裙,23.0,106.0,430.0,696.0,1,train


In [ ]:
dpedia_catg23[dpedia_catg23['']]

In [18]:
!tree -L 1 $info_root

/home/chenyi/workspace/dataset/deepedia/info
├── cat_pedia2ours.pickle
├── fashion2info_train_pd.pickle
├── fashion2info_val_pd.pickle
├── id2name.pickle
├── pedia2yolo_train.pickle
├── pedia2yolo_val.pickle
├── pedia_info_pd.pickle
└── pedia_train_dict.pickle

0 directories, 8 files


In [6]:
fashion2_ours = \
            {0: 4, 1: 4,  2:4, 3:4, 4:4, 5:4,  6:5,  7:5,  8:6,  9:7, 10:7,  11:7,  12:7}

In [166]:
with open(f'{root}/info/fashion2info_val_pd.pickle', 'rb') as f:
    data_pd = pickle.load(f)

In [167]:
data_pd.head()

,path,id,bbox,label_path,img_path
0,/Users/chenyi/Desktop/code/dataset/deepfashion...,019065,"[[7.0, 0.6474359, 0.7685185, 0.6581197, 0.4601...",/home/chenyi/workspace/dataset/deepedia/labels...,/home/chenyi/workspace/dataset/deepedia/images...
1,/Users/chenyi/Desktop/code/dataset/deepfashion...,016356,"[[11.0, 0.54, 0.5800395, 0.176, 0.4486166]]",/home/chenyi/workspace/dataset/deepedia/labels...,/home/chenyi/workspace/dataset/deepedia/images...
2,/Users/chenyi/Desktop/code/dataset/deepfashion...,011439,"[[0.0, 0.52136755, 0.6448664, 0.8333333, 0.687...",/home/chenyi/workspace/dataset/deepedia/labels...,/home/chenyi/workspace/dataset/deepedia/images...
3,/Users/chenyi/Desktop/code/dataset/deepfashion...,009276,"[[10.0, 0.49786323, 0.5608974, 0.99572647, 0.7...",/home/chenyi/workspace/dataset/deepedia/labels...,/home/chenyi/workspace/dataset/deepedia/images...
4,/Users/chenyi/Desktop/code/dataset/deepfashion...,026295,"[[8.0, 0.5117521, 0.47079772, 0.9465812, 0.551...",/home/chenyi/workspace/dataset/deepedia/labels...,/home/chenyi/workspace/dataset/deepedia/images...


In [168]:
fashion2info_pd = data_pd

In [169]:
# %%time
# maping to model catgory id.
bboxes_newlist = []
for ind,row in tqdm(fashion2info_pd.iterrows()):
    bboxes = row['bbox']
    bboxes_list = []
    for bx in bboxes:
        id_ = int(bx[0])
        id_new = fashion2_ours[id_]
        bx[0] = int(id_new)
        bboxes_list.append(bx)
    bboxes_newlist.append(bboxes_list)

32153it [00:01, 31652.25it/s]


In [170]:
data_pd['bbox'] = bboxes_newlist

In [171]:
data_pd['label_path'][:2].values

array(['/home/chenyi/workspace/dataset/deepedia/labels/val/019065.txt',
       '/home/chenyi/workspace/dataset/deepedia/labels/val/016356.txt'],
      dtype=object)

In [172]:
# data_pd['label_path'] =  data_pd['label_path'].apply(lambda x: x.replace('/labels/', '/labels_catg6/'))

In [173]:
for ind,row in tqdm(fashion2info_pd.iterrows()):
    bboxes = row['bbox']
    label_p = row['label_path']
    text = bboxes2string(bboxes)
    write_bboxes(label_p, text)
    # break

32153it [00:02, 11019.23it/s]


### pedia的label问题

In [ ]:
with open('/home/chenyi/workspace/dataset/deepedia/info/pedia_train_dict.pickle', 'rb') as f:
    pedia_info = pickle.load(f)

In [ ]:
with open('/home/chenyi/workspace/dataset/deepedia/info/pedia_info_pd.pickle', 'rb') as f:
    pedia_info_pd = pickle.load(f)

In [ ]:
train_root = '/home/chenyi/workspace/dataset/data_fashionpedia/train/'

In [ ]:
pedia_info_pd['img_path'] = train_root + pedia_info_pd['file_name'].values

In [ ]:
pedia_info_pd.head()

In [ ]:
ind = 20113
row_pd = pedia_info_pd.iloc[ind,:]

In [ ]:
img_np = cv2.imread(row_pd['img_path'])
h,w,c = img_np.shape
row = row_pd['bbox'][3]
id_ = row[0]
box_coco = row[1:]
box_item = []
for item in box_coco:
    box_item.append(int(item))
bbox_item = bbox_coco2voc(box_item)
img_np = bbox_draw(img_np, box_item)
Image.fromarray(img_np)

### 写操作

In [122]:
def write_bboxes_text(bboxes, p):
    bboxes_str = ''
    for bx in bboxes:
        bboxes_str += f'{str(int(bx[0]))} \
                        {str(bx[1])}  \
                        {str(bx[2])} \
                        {str(bx[3])} \
                        {str(bx[4])} \n'
            
    with open(p, 'w') as f:
        f.write(bboxes_str)

In [191]:
with open('/home/chenyi/workspace/dataset/deepedia/info/pedia_train_dict.pickle', 'rb') as f:
    data_dict = pickle.load(f)
with open(f'{root}/info/pedia_info_pd.pickle', 'rb') as f:
    data_pd = pickle.load(f)
with open(f'{root}/info/cat_pedia2ours.pickle', 'rb') as f:
    id2name = pickle.load(f)
data_pd['bbox'] = data_dict['bbox']
pedia_info_pd = data_pd

In [192]:
pedia_info_pd.head()

,img_id,bbox,width,height,file_name,train
0,46226,"[[6.0, 228.0, 771.0, 329.0, 252.0], [0.0, 237....",721,1024,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,train
1,29130,"[[33.0, 376.0, 170.0, 254.0, 108.0], [1.0, 229...",1024,1024,0000fe7c9191fba733c8a69cfaf962b7.jpg,train
2,32175,"[[33.0, 303.0, 245.0, 83.0, 32.0], [10.0, 259....",681,1024,0002ec21ddb8477e98b2cbb87ea2e269.jpg,train
3,43328,"[[10.0, 0.0, 202.0, 720.0, 820.0], [33.0, 341....",722,1024,0002f5a0ebc162ecfb73e2c91e3b8f62.jpg,val
4,38187,"[[10.0, 240.0, 256.0, 371.0, 717.0], [33.0, 33...",682,1024,0004467156e47b0eb6de4aa6479cbd15.jpg,train


In [193]:
id2name

{19: 0,
 14: 1,
 23: 2,
 24: 3,
 0: 4,
 1: 4,
 2: 4,
 3: 4,
 4: 4,
 5: 4,
 6: 5,
 7: 5,
 8: 6,
 9: 7,
 10: 7,
 11: 7}

In [194]:
# id2name[9] = 4
# del id2name[14]
# del id2name[19]

In [195]:
id2name

{23: 2,
 24: 3,
 0: 4,
 1: 4,
 2: 4,
 3: 4,
 4: 4,
 5: 4,
 6: 5,
 7: 5,
 8: 6,
 9: 4,
 10: 7,
 11: 7}

In [196]:
%%time
ks = id2name.keys()
bboxes_list = []
for ind, row in pedia_info_pd.iterrows():
    bboxes = row['bbox']
    w,h = row['width'], row['height']
    bx_new = []
    for bx in bboxes:
        k_new = int(bx[0])
        if k_new in ks:
            bx[0] = id2name[k_new] - 2
            bx[1:] = bbox_coco2voc(bx[1:])
            bx[1:] = bbox_voc2yolo(bx[1:], w, h)
            bx_new.append(bx)
    bboxes_list.append(bx_new)

CPU times: user 2 s, sys: 400 µs, total: 2 s
Wall time: 2 s


In [197]:
pedia_info_pd['bbox'] = bboxes_list

In [198]:
pedia_info_pd.head()

,img_id,bbox,width,height,file_name,train
0,46226,"[[3.0, 0.544382801664355, 0.8759765625, 0.4563...",721,1024,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,train
1,29130,"[[2.0, 0.50927734375, 0.544921875, 0.571289062...",1024,1024,0000fe7c9191fba733c8a69cfaf962b7.jpg,train
2,32175,"[[5.0, 0.5058737151248165, 0.447265625, 0.2511...",681,1024,0002ec21ddb8477e98b2cbb87ea2e269.jpg,train
3,43328,"[[5.0, 0.4986149584487535, 0.59765625, 0.99722...",722,1024,0002f5a0ebc162ecfb73e2c91e3b8f62.jpg,val
4,38187,"[[5.0, 0.623900293255132, 0.60009765625, 0.543...",682,1024,0004467156e47b0eb6de4aa6479cbd15.jpg,train


In [199]:
# !cp -r /home/chenyi/workspace/dataset/deepedia/labels /home/chenyi/workspace/dataset/deepedia/labels_catg6

In [200]:
mode = 'val'
pedia_train = pedia_info_pd[pedia_info_pd['train']==mode]

In [201]:
# train_t_root = '/home/chenyi/workspace/dataset/deepedia/images/val/'
label_t_root = f'/home/chenyi/workspace/dataset/deepedia/labels_catg6/{mode}/'
# pedia_train['img_path'] = train_s_root +  pedia_train['file_name'].values
# pedia_train['label_path'] = train_s_root +  pedia_train['file_name'].values
# pedia_train['label_target'] = pedia_train['label_target'].apply(
#     lambda x:x.replace('.jpg', '.txt')
# )

In [202]:
pedia_train['label_tp'] = pedia_train['file_name'].apply(
    lambda x:f'{label_t_root}{x}'
)

/tmp/ipykernel_403767/360313808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedia_train['label_tp'] = pedia_train['file_name'].apply(


In [203]:
pedia_train['label_tp'] = pedia_train['label_tp'].apply(
    lambda x:x.replace('.jpg', '.txt')
)

/tmp/ipykernel_403767/1035783826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedia_train['label_tp'] = pedia_train['label_tp'].apply(


In [204]:
pedia_train['label_tp'][:2].values

array(['/home/chenyi/workspace/dataset/deepedia/labels_catg6/val/0002f5a0ebc162ecfb73e2c91e3b8f62.txt',
       '/home/chenyi/workspace/dataset/deepedia/labels_catg6/val/00048c3a2fb9c29340473c4cfc06424a.txt'],
      dtype=object)

In [205]:
import shutil

In [206]:
for ind,row in tqdm(pedia_train.iterrows()):
    bboxes = row['bbox']
    # img_s_path = row['img_path']
    # img_t_path = row['img_target']
    label_path = row['label_tp']
    write_bboxes_text(bboxes, label_path)
    # shutil.copyfile(img_s_path, img_t_path)
    # break

9087it [00:00, 10946.64it/s]


### 软链接

In [ ]:
# !mkdir /home/chenyi/workspace/yolov5/datasets/deepedia_catg6/

In [207]:
!tree -d -L 2 /home/chenyi/workspace/yolov5/datasets/deepedia_catg6/

/home/chenyi/workspace/yolov5/datasets/deepedia_catg6/
├── images
│   ├── train -> /home/chenyi/workspace/dataset/deepedia/images/train/
│   └── val -> /home/chenyi/workspace/dataset/deepedia/images/val/
└── labels
    ├── train -> /home/chenyi/workspace/dataset/deepedia/labels_catg6/train/
    └── val -> /home/chenyi/workspace/dataset/deepedia/labels_catg6/val/

6 directories


In [156]:
!ln -s /home/chenyi/workspace/dataset/deepedia/labels_catg6/val/ /home/chenyi/workspace/yolov5/datasets/deepedia_catg6/labels/

In [151]:
# !unlink  /home/chenyi/workspace/yolov5/datasets/deepedia_catg6/images/train